In [ ]:
# Google Earth Engine packages
import ee
import geemap

# other packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# GIS packages
from pysheds.grid import Grid
import fiona


#Define a function to plot the digital elevation model
def plotFigure(data, label, cmap='Blues'):
    plt.figure(figsize=(12,10))
    plt.imshow(data, extent=grid.extent)
    plt.colorbar(label=label)
    plt.grid()

In [ ]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()

In [ ]:
# get config data
filename = 'output/srtm_gee.tif'
output_file = 'output/srtm_gee.shp'

#dem = ['CGIAR/SRTM90_V4','elevation','SRTM 90m']
dem = ['USGS/SRTMGL1_003','elevation','SRTM NASA 30m']
#dem = ['MERIT/DEM/v1_0_3','dem', 'MERIT 30m']

y, x = 42.30029106, 78.09146228 # hydro new

In [ ]:
# call map that centers aroung the region
Map = geemap.Map()
Map

In [ ]:
image = ee.Image(dem[0])
srtm_vis = { 'bands': dem[1],
             'min': 0,
             'max': 6000,
            'palette': ['000000', '478FCD', '86C58E', 'AFC35E', '8F7131','B78D4F', 'E2B8A6', 'FFFFFF']
           }
Map.addLayer(image, srtm_vis, dem[2], True, 0.7)

In [ ]:
point = ee.Geometry.Point(x,y)
Map.addLayer(point,{'color': 'blue'},'Discharge Point');

box = point.buffer(40000).bounds()
Map.addLayer(box,{'color': 'grey'},'Catchment Area', True, 0.7);
Map.centerObject(box, zoom=9)

In [ ]:
feature = Map.draw_last_feature
if feature is not None:
    box = feature.geometry()
    print("Manually drawn box will be considered")

In [ ]:
geemap.ee_export_image(image, filename=filename, scale=30, region=box, file_per_band=False)

In [ ]:
DEM_file = filename

# Plot the DEM
grid = Grid.from_raster(DEM_file)
dem = grid.read_raster(DEM_file)
grid.view(dem)

# Fill depressions in DEM
flooded_dem = grid.fill_depressions(dem)
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)

# Specify directional mapping
#N    NE    E    SE    S    SW    W    NW
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
# Compute flow directions
fdir = grid.flowdir(inflated_dem, dirmap=dirmap)

catch = grid.catchment(x=x, y=y, fdir=fdir, dirmap=dirmap, xytype='coordinate')

# Clip the DEM to the catchment
grid.clip_to(catch)
clipped_catch = grid.view(catch)

In [ ]:
demView = grid.view(dem, nodata=np.nan)
plotFigure(demView,'Elevation')
plt.show()

In [ ]:
## Create shapefile and save it
shapes = grid.polygonize()

schema = {
    'geometry': 'Polygon',
    'properties': {'LABEL': 'float:16'}
}

with fiona.open(output_file, 'w',
                driver='ESRI Shapefile',
                crs=grid.crs.srs,
                schema=schema) as c:
    i = 0
    for shape, value in shapes:
        rec = {}
        rec['geometry'] = shape
        rec['properties'] = {'LABEL' : str(value)}
        rec['id'] = str(i)
        c.write(rec)
        i += 1

In [ ]:
print("Mean catchment elevation is " + str(np.nanmean(demView)) + " m")